# **AutoGen - Equipos de Agentes (Teams)**

Un **team** en AutoGen es un grupo de agentes que trabajan juntos para lograr un objetivo común. Se recomienda usar equipos cuando una sola IA no es suficiente para resolver una tarea y se requiere **colaboración y especialización**.

## **¿Cuándo usar un equipo de agentes?**

✅ Útil para tareas complejas que requieren colaboración.  
❌ No recomendable para tareas simples que un solo agente puede manejar.  
📌 **Recomendación:** Antes de crear un equipo, optimiza un solo agente con las herramientas adecuadas.

---

## **1️⃣ Creando un equipo con RoundRobinGroupChat**

La estructura **RoundRobinGroupChat** es un equipo donde los agentes **comparten el mismo contexto** y **se turnan para responder** de forma ordenada.

📌 **Ejemplo: Equipo con dos agentes (Asistente + Crítico)**  
Este equipo usa el **patrón de reflexión**, donde un crítico evalúa las respuestas del asistente.

### **Paso 1: Importar las librerías**

In [5]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os
from autogen_core.models import UserMessage


### **Paso 2: Crear los agentes**

In [6]:

# Crear un cliente de modelo OpenAI
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    api_key= os.getenv("OPENAI_API_KEY")
)

# Agente principal que genera contenido
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant."
)

# Agente crítico que revisa el contenido y da feedback
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' when your feedback is addressed."
)

### **Paso 3: Definir una condición de terminación**


In [7]:
# Condición de terminación: el equipo se detendrá cuando el crítico diga "APPROVE"
text_termination = TextMentionTermination("APPROVE")

### **Paso 4: Crear el equipo**

In [9]:
# Configurar el equipo con los agentes y la condición de terminación
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

---

## **2️⃣ Ejecutando un equipo**

Podemos ejecutar el equipo con `run()` para completar una tarea de manera estructurada.

### **Ejemplo: Escribir un poema sobre el otoño** 

In [10]:
# Ejecutar el equipo con una tarea
result = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=131), content="Golden leaves begin to dance,  \nIn the cool and gentle breeze,  \nWhispering secrets of romance,  \nAmongst the aging trees.  \n\nThe air is crisp, the skies are bright,  \nPumpkin scents drift far and wide,  \nNature dons her cloak of light,  \nIn hues that cannot hide.  \n\nFields of amber, skies of gray,  \nThe world slows down its pace,  \nAs autumn gently paves the way,  \nFor winter's cold embrace.  \n\nIn each moment rich and true,  \nAutumn's charm is found,  \nA fleeting time, a wondrous hue,  \nWhere beauty does abound.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=174, completion_tokens=166), content='This is a beautiful and evocative poem that captures the essence of the fall season

In [11]:
# Reiniciar el equipo antes de una nueva tarea
await team.reset()

# Ejecutar con streaming para ver cada mensaje a medida que se genera
async for message in team.run_stream(task="Write a short poem about the fall season."):
    print(message)

source='user' models_usage=None content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=102) content="Golden leaves in gentle flight,  \nWhispering winds in the crisp twilight.  \nPumpkin patches, fields aglow,  \nHarvest moons that softly show.  \n\nSweaters wrapped in warm embrace,  \nThe cozy hearth, a glowing space.  \nNature's palette, rich and bold,  \nStories by the fire told.  \n\nIn the stillness, beauty calls,  \nEchoes of the fragrant fall.  \nA fleeting dance 'neath amber skies,  \nWhere time slows down and magic lies.  " type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=145, completion_tokens=124) content="This is a lovely poem that captures the essence of the fall season beautifully. The imagery of golden leaves, whispering winds, and harvest moons creates a vivid picture of autumnal beauty. The lines about sweaters, a cozy hearth, and stories by the fire